# Contextual Chunk Headers (CCH)

## Tổng quan

Tiêu đề chunk theo ngữ cảnh (Contextual Chunk Headers - CCH) là một phương pháp tạo tiêu đề chunk chứa ngữ cảnh cấp cao hơn (ví dụ: ngữ cảnh cấp tài liệu hoặc cấp phần), và thêm các tiêu đề chunk đó vào đầu các chunk trước khi nhúng chúng. Điều này cung cấp cho các embedding một biểu diễn chính xác và đầy đủ hơn về nội dung và ý nghĩa của văn bản. Trong thử nghiệm của chúng tôi, tính năng này dẫn đến sự cải thiện đáng kể về chất lượng truy xuất. Ngoài việc tăng tỷ lệ truy xuất thông tin chính xác, CCH còn giảm tỷ lệ kết quả không liên quan xuất hiện trong kết quả tìm kiếm. Điều này làm giảm tỷ lệ LLM hiểu sai một đoạn văn bản trong các ứng dụng trò chuyện và tạo nội dung downstream.

## Động lực

Nhiều vấn đề mà các nhà phát triển gặp phải với RAG (Retrieval-Augmented Generation) đều quy về điều này: Các chunk riêng lẻ thường không chứa đủ ngữ cảnh để được hệ thống truy xuất hoặc LLM sử dụng đúng cách. Điều này dẫn đến việc không thể trả lời câu hỏi và, đáng lo ngại hơn, là ảo giác (hallucinations).

Ví dụ về vấn đề này:
- Các chunk thường đề cập đến chủ đề của chúng thông qua các tham chiếu và đại từ ngầm định. Điều này khiến chúng không được truy xuất khi cần thiết, hoặc không được LLM hiểu đúng cách.
- Các chunk riêng lẻ thường chỉ có ý nghĩa trong ngữ cảnh của toàn bộ phần hoặc tài liệu, và có thể gây hiểu lầm khi đọc riêng lẻ.

## Các thành phần chính

#### Tiêu đề chunk theo ngữ cảnh (Contextual Chunk Headers)
Ý tưởng ở đây là thêm ngữ cảnh cấp cao hơn vào chunk bằng cách thêm tiêu đề chunk vào đầu. Tiêu đề chunk này có thể đơn giản chỉ là tiêu đề tài liệu, hoặc có thể sử dụng kết hợp tiêu đề tài liệu, tóm tắt ngắn gọn tài liệu và toàn bộ hệ thống phân cấp tiêu đề phần và tiêu đề phần phụ.

## Chi tiết phương pháp

#### Tạo ngữ cảnh
Trong bản trình diễn dưới đây, chúng tôi sử dụng LLM để tạo tiêu đề mô tả cho tài liệu. Điều này được thực hiện thông qua một prompt đơn giản, trong đó bạn truyền một phiên bản rút gọn của văn bản tài liệu và yêu cầu LLM tạo tiêu đề mô tả cho tài liệu. Nếu bạn đã có tiêu đề tài liệu mô tả đầy đủ, bạn có thể sử dụng trực tiếp chúng. Chúng tôi nhận thấy rằng tiêu đề tài liệu là loại ngữ cảnh cấp cao đơn giản và quan trọng nhất để đưa vào tiêu đề chunk.

Các loại ngữ cảnh khác bạn có thể đưa vào tiêu đề chunk:
- Tóm tắt ngắn gọn tài liệu
- Tiêu đề phần/phần phụ
    - Điều này giúp hệ thống truy xuất xử lý các truy vấn cho các phần hoặc chủ đề lớn hơn trong tài liệu.

#### Nhúng các chunk với tiêu đề chunk
Văn bản bạn nhúng cho mỗi chunk chỉ đơn giản là sự nối của tiêu đề chunk và văn bản chunk. Nếu bạn sử dụng reranker trong quá trình truy xuất, bạn sẽ muốn đảm bảo rằng bạn cũng sử dụng sự nối này ở đó.

#### Thêm tiêu đề chunk vào kết quả tìm kiếm
Bao gồm tiêu đề chunk khi trình bày kết quả tìm kiếm cho LLM cũng có lợi vì nó cung cấp cho LLM nhiều ngữ cảnh hơn và giảm khả năng LLM hiểu sai ý nghĩa của một chunk.


![Your Technique Name](images/contextual_chunk_headers.svg)

In [3]:
# Basic setup
import os
import tiktoken
from typing import List
from langchain_ollama import ChatOllama
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader


llm_model_name='qwen2.5:7b'
llm = ChatOllama(model=llm_model_name, temperature=0)

embedding_model='bge-m3:latest'
embeddings = OllamaEmbeddings(model=embedding_model)

In [ ]:
def replace_t_with_space(list_of_documents):
    """
    Thay thế tất cả các ký tự tab ('\t') bằng dấu cách trong nội dung trang của mỗi tài liệu

    Args:
        list_of_documents: Danh sách các đối tượng tài liệu, mỗi đối tượng có thuộc tính 'page_content'.

    Returns:
        Danh sách tài liệu đã được sửa đổi với các ký tự tab được thay thế bằng dấu cách.
    """

    for doc in list_of_documents:
        doc.page_content = doc.page_content.replace('\t', ' ')  # Replace tabs with spaces
    return list_of_documents

def pretty_print_docs(docs):
    # Print each document in the list with a separator between them
    print(
        f"\n{'-' * 100}\n".join(  # Separator line for better readability
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]  # Format: Document number + content
        )
    )
  
def split_into_chunks(path: str, chunk_size: int = 1000) -> list[str]:
  # Load PDF documents
  loader = PyPDFLoader(path)
  documents = loader.load()
  
  # Split documents into chunks
  text_splitter = RecursiveCharacterTextSplitter(
      chunk_size=chunk_size, chunk_overlap=0, length_function=len
  )
  texts = text_splitter.split_documents(documents)
  cleaned_texts = replace_t_with_space(texts)
  
  return [doc.page_content for doc in cleaned_texts]


FILE_PATH='law_gtdb/data/law_gtdb_35.pdf'

chunks = split_into_chunks(FILE_PATH)  

In [5]:
chunks[0]

'CÔNG BÁO/Số 983 + 984/Ngày 25-8-2024 3 \n \n \nQUỐC HỘI \n \nCỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM \nĐộc lập - Tự do - Hạnh phúc \nLuật số: 35/2024/QH15  \n \nLUẬT \nĐƯỜNG BỘ \n \nCăn cứ Hiến pháp nước Cộng hòa xã hội chủ nghĩa Việt Nam; \nQuốc hội ban hành Luật Đường bộ. \n \nChương I \nNHỮNG QUY ĐỊNH CHUNG \n \nĐiều 1. Phạm vi điều chỉnh  \nLuật này quy định về hoạt động đường bộ và quản lý nhà nước về hoạt động \nđường bộ.  \nĐiều 2. Giải thích từ ngữ  \nTrong Luật này, các từ ngữ dưới đây được hiểu như sau: \n1. Hoạt động đường bộ bao gồm: hoạt động về quy hoạch, đầu tư, xây dựng, \nquản lý, sử dụng, vận hành, khai thác, bảo trì, bảo vệ kết cấu hạ tầng đường bộ; \nvận tải đường bộ.  \n2. Đường bộ bao gồm: đường, cầu đường bộ, cống đường bộ, hầm đường bộ, \nbến phà đường bộ, cầu phao đường bộ, đường cứu nạn và các công trình phụ trợ \ngắn liền với đường bộ. \n3. Công trình đường bộ bao gồm: đường bộ; công trình phục vụ công tác quản \nlý, vận hành, khai thác đường bộ; công trình thuộc

### Generate descriptive document title to use in chunk header

In [21]:
# Constants
DOCUMENT_TITLE_PROMPT = """
INSTRUCTIONS
What is the title of the following document in Vietnamese?

Your response MUST be the title of the document, and nothing else. DO NOT respond with anything else.

{document_title_guidance}

{truncation_message}

DOCUMENT
{document_text}
""".strip()

TRUNCATION_MESSAGE = """
Also note that the document text provided below is just the first ~{num_words} words of the document. That should be plenty for this task. Your response should still pertain to the entire document, not just the text provided below.
""".strip()

MAX_CONTENT_TOKENS = 4000
TOKEN_ENCODER = tiktoken.encoding_for_model('gpt-3.5-turbo')

def make_llm_call(chat_messages: list[dict]) -> str:
  model = llm = ChatOllama(model=llm_model_name, temperature=0.2, num_predict=MAX_CONTENT_TOKENS)
  
  response = model.invoke(chat_messages)
  return response.content

def truncate_content(content: str, max_tokens: int) -> tuple[str, int]:
  tokens = TOKEN_ENCODER.encode(content, disallowed_special=())
  truncated_tokens = tokens[:max_tokens]
  return TOKEN_ENCODER.decode(truncated_tokens), min(len(tokens), max_tokens)

def get_document_title(document_text: str, document_title_guidance: str = "") -> str:
  # Truncate the content if it's too long
  document_text, num_tokens = truncate_content(document_text, MAX_CONTENT_TOKENS)
  truncation_message = TRUNCATION_MESSAGE.format(num_words=3000) if num_tokens >= MAX_CONTENT_TOKENS else ""

  # Prepare the prompt for title extraction
  prompt = DOCUMENT_TITLE_PROMPT.format(
      document_title_guidance=document_title_guidance,
      document_text=document_text,
      truncation_message=truncation_message
  )
  chat_messages = [{"role": "user", "content": prompt}]
  
  return make_llm_call(chat_messages)

In [22]:
chunks[0]

'CÔNG BÁO/Số 983 + 984/Ngày 25-8-2024 3 \n \n \nQUỐC HỘI \n \nCỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM \nĐộc lập - Tự do - Hạnh phúc \nLuật số: 35/2024/QH15  \n \nLUẬT \nĐƯỜNG BỘ \n \nCăn cứ Hiến pháp nước Cộng hòa xã hội chủ nghĩa Việt Nam; \nQuốc hội ban hành Luật Đường bộ. \n \nChương I \nNHỮNG QUY ĐỊNH CHUNG \n \nĐiều 1. Phạm vi điều chỉnh  \nLuật này quy định về hoạt động đường bộ và quản lý nhà nước về hoạt động \nđường bộ.  \nĐiều 2. Giải thích từ ngữ  \nTrong Luật này, các từ ngữ dưới đây được hiểu như sau: \n1. Hoạt động đường bộ bao gồm: hoạt động về quy hoạch, đầu tư, xây dựng, \nquản lý, sử dụng, vận hành, khai thác, bảo trì, bảo vệ kết cấu hạ tầng đường bộ; \nvận tải đường bộ.  \n2. Đường bộ bao gồm: đường, cầu đường bộ, cống đường bộ, hầm đường bộ, \nbến phà đường bộ, cầu phao đường bộ, đường cứu nạn và các công trình phụ trợ \ngắn liền với đường bộ. \n3. Công trình đường bộ bao gồm: đường bộ; công trình phục vụ công tác quản \nlý, vận hành, khai thác đường bộ; công trình thuộc

In [ ]:
# def rerank_documents(query: str, chunks: List[str]) -> List[float]:
question = "Quy định đặt tên đường bộ là gì?"
